In [3]:
import pandas as pd 
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text

In [4]:
# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///app.db', echo=True)

In [12]:
create_creators_table = text('''
CREATE TABLE IF NOT EXISTS creators (
    creator_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE NOT NULL,
    name VARCHAR(255) NOT NULL,
    bio TEXT,
    social_media_links TEXT,  -- JSON or serialized text format to store multiple links
    specialties TEXT,  -- Could be a list of specialties or focus areas
    audience_size INTEGER,
    contact_email VARCHAR(255),
    website URL,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);
''')

# Assuming 'engine' is already defined and imported from SQLAlchemy setup
with engine.connect() as connection:
    connection.execute(create_creators_table)
    print("Creators table created successfully.")


2024-04-17 11:15:58,923 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 11:15:58,924 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS creators (
    creator_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER UNIQUE NOT NULL,
    name VARCHAR(255) NOT NULL,
    bio TEXT,
    social_media_links TEXT,  -- JSON or serialized text format to store multiple links
    specialties TEXT,  -- Could be a list of specialties or focus areas
    audience_size INTEGER,
    contact_email VARCHAR(255),
    website URL,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);

2024-04-17 11:15:58,924 INFO sqlalchemy.engine.Engine [cached since 464.3s ago] ()
Creators table created successfully.
2024-04-17 11:15:58,924 INFO sqlalchemy.engine.Engine ROLLBACK


In [42]:
def add_creator(**kwargs):
    columns = ', '.join(kwargs.keys())
    values = ', '.join([f":{key}" for key in kwargs.keys()])
    insert_query = text(f"""
        INSERT INTO creators ({columns})
        VALUES ({values});
    """)

    with engine.begin() as connection:
        connection.execute(insert_query, kwargs)
        print(f"Creator added successfully with details: {kwargs}")


def modify_creator(creator_id, **kwargs):
    updates = [f"{key} = :{key}" for key in kwargs]
    update_query = text(f"UPDATE creators SET {', '.join(updates)} WHERE creator_id = :creator_id")

    kwargs['creator_id'] = creator_id  # Ensure the creator_id is included for the WHERE clause

    with engine.begin() as connection:
        result = connection.execute(update_query, kwargs)
        if result.rowcount > 0:
            print(f"Creator with ID {creator_id} updated successfully.")
        else:
            print(f"No creator found with ID {creator_id}, nothing updated.")



def delete_creator(creator_id):
    delete_query = text("DELETE FROM creators WHERE creator_id = :creator_id")

    with engine.begin() as connection:
        result = connection.execute(delete_query, {'creator_id': creator_id})
        if result.rowcount > 0:
            print(f"Creator with ID {creator_id} deleted successfully.")
        else:
            print(f"No creator found with ID {creator_id}, nothing deleted.")



def get_creator(**kwargs):
    if not kwargs:
        print("No parameters provided to search for creator.")
        return None

    conditions = [f"{key} = :{key}" for key in kwargs]
    where_clause = " AND ".join(conditions)
    select_query = text(f"SELECT * FROM creators WHERE {where_clause}")

    print(select_query)
    with engine.connect() as connection:
        result = connection.execute(select_query, kwargs).fetchone()
        print(result)
        
        if result:
            creator_dict = {
            'creator_id':result[0],
            'user_id':result[1],
            'name':result[2],
            'bio':result[3],
            'social_media_links':result[4],
            'specialties':result[5],
            'audience_size':result[6],
            'contact_email':result[7],
            'website':result[8]
            }
            
            return creator_dict
        else:
            print("Creator not found.")
            return None

In [72]:
add_creator(user_id = 1,name='Cyprien')

2024-04-17 11:39:02,580 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 11:39:02,581 INFO sqlalchemy.engine.Engine 
        INSERT INTO creators (user_id, name)
        VALUES (?, ?);
    
2024-04-17 11:39:02,582 INFO sqlalchemy.engine.Engine [cached since 1605s ago] (1, 'Cyprien')
Creator added successfully with details: {'user_id': 1, 'name': 'Cyprien'}
2024-04-17 11:39:02,584 INFO sqlalchemy.engine.Engine COMMIT


In [76]:
pd.read_sql('SELECT * FROM creators',engine)

2024-04-17 11:39:10,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 11:39:10,494 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM creators")
2024-04-17 11:39:10,494 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 11:39:10,495 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM creators")
2024-04-17 11:39:10,495 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 11:39:10,496 INFO sqlalchemy.engine.Engine SELECT * FROM creators
2024-04-17 11:39:10,496 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-17 11:39:10,497 INFO sqlalchemy.engine.Engine ROLLBACK


,creator_id,user_id,name,bio,social_media_links,specialties,audience_size,contact_email,website


In [74]:
user_id = 1
creator_id = get_creator(user_id=user_id)['creator_id']

print(creator_id)
modify_creator(creator_id=creator_id, bio = '''Hello je m'appelle Cyprien les gars''')

SELECT * FROM creators WHERE user_id = :user_id
2024-04-17 11:39:06,140 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 11:39:06,141 INFO sqlalchemy.engine.Engine SELECT * FROM creators WHERE user_id = ?
2024-04-17 11:39:06,141 INFO sqlalchemy.engine.Engine [cached since 1111s ago] (1,)
(7, 1, 'Cyprien', None, None, None, None, None, None)
2024-04-17 11:39:06,142 INFO sqlalchemy.engine.Engine ROLLBACK
7
2024-04-17 11:39:06,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 11:39:06,143 INFO sqlalchemy.engine.Engine UPDATE creators SET bio = ? WHERE creator_id = ?
2024-04-17 11:39:06,144 INFO sqlalchemy.engine.Engine [cached since 1282s ago] ("Hello je m'appelle Cyprien les gars", 7)
Creator with ID 7 updated successfully.
2024-04-17 11:39:06,146 INFO sqlalchemy.engine.Engine COMMIT


In [75]:
delete_creator(creator_id = creator_id)

2024-04-17 11:39:07,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-17 11:39:07,919 INFO sqlalchemy.engine.Engine DELETE FROM creators WHERE creator_id = ?
2024-04-17 11:39:07,920 INFO sqlalchemy.engine.Engine [cached since 1349s ago] (7,)
Creator with ID 7 deleted successfully.
2024-04-17 11:39:07,921 INFO sqlalchemy.engine.Engine COMMIT
